In [ ]:
# %pip install hvplot
# %pip install plotly
# %pip install holoviews
# %pip install xarray
# %pip install panel

In [ ]:
import numpy as np

import datetime as dt
import panel as pn

pn.extension()
import panel as pn
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

import pathlib

import holoviews as hv
import matplotlib.pyplot as plt
import hvplot
import param
import hvplot.pandas


pn.extension(sizing_mode='stretch_width')

pn.extension('perspective')

In [ ]:
# Contact Project Administrator to get these
TENANT_ID = "3aa4a235-b6e2-48d5-9195-7fcf05b459b0" # AAD using @equinor.com
CLIENT_ID = "3d7a85da-05ab-4675-b5f9-0c4b96a1c84b" # CDF_ALL_USER_APPLICATION_FLOW
CDF_CLUSTER = "westeurope-1"

# -------------for DEV project-----------------
COGNITE_PROJECT = "equinor-dev"

# -------------for TEST project-----------------
# COGNITE_PROJECT = "equinor-test"

In [ ]:
from cognite.client import CogniteClient
client = CogniteClient.default_oauth_interactive(
    project=COGNITE_PROJECT,
    tenant_id=TENANT_ID,
    client_id=CLIENT_ID,
    cdf_cluster=CDF_CLUSTER,
    client_name="my-cognite-python-test", # a name to identify your session
)
from cognite.client.data_classes import LabelFilter

In [ ]:
display(f'{client.version=} / {client.config.project=}')
# print(client.iam.token.inspect())

In [ ]:
#test

In [ ]:
wellcom_dataset = "src:015:wellcom"
edm_dataset = "src:012:edm"
witsml_dataset = "src:014:witsmldata"

In [ ]:
active_wellbore = "active:wellbore"
match_witsml = "match:witsml:wellcom" # sensor data only for johan svendrup
match_edm = "match:edm:wellcom"

# defien and show datasets

In [ ]:
#list wellcom wellbores that are active in WITSML
active_wellbores = client.assets.list(data_set_external_ids = wellcom_dataset, labels=LabelFilter(contains_all=[active_wellbore]), limit=-1)
active_wellbores

In [ ]:
witsml_db  = client.time_series.list(data_set_external_ids = witsml_dataset, limit=-1)
witsml_db

## make to padnas

In [ ]:
pd_active_wellbores = active_wellbores.to_pandas(metadata_prefix=active_wellbores)
pd_active_wellbores

In [ ]:
_pd_witsml = witsml_db.to_pandas(metadata_prefix=witsml_db)
_pd_witsml

### query pandas witsml

In [ ]:
_pd_witsml.columns

In [ ]:
import json 
import pandas as pd 
# from pandas.json_normalize import json_normalize #package for flattening json in pandas df

## flatten witsml data

In [ ]:
_pd_witsml_flattened = pd.concat([_pd_witsml.drop(['metadata'], axis=1), pd.json_normalize(_pd_witsml['metadata'])], axis=1)



In [ ]:
_pd_witsml_flattened.columns

In [ ]:
_pd_witsml_flattened = _pd_witsml_flattened[['id', 'uidWell', 'uidWellbore',  'nameWellbore','external_id', 'name', 'is_string', 'unit', 'asset_id', 'is_step',
       'description', 'security_categories', 'data_set_id', 'created_time',
       'last_updated_time', 'type', 'commonData_dTimCreation',
       'commonData_dTimLastChange', 'commonData_itemState',
       'commonData_sourceName', 'creationDate', 'direction',
       'endDateTimeIndex', 'indexCurve', 'indexType', 'name', 'nameWell',
       'nullValue', 'objectGrowing', 'serviceCompany',
       'startDateTimeIndex',  'uidLog',
       'unavailable_in_source', 'witsml-server-ref', 'source',
       'curveDescription', 'maxDateTimeIndex', 'minDateTimeIndex', 'mnemAlias',
       'mnemonic', 'typeLogData', 'uid', 'unit']]
_pd_witsml_flattened

In [ ]:
_pd_witsml_flattened[_pd_witsml_flattened['description'] == 'BITDEP']['external_id'].iloc[0]

In [ ]:
_pd_witsml_flattened['nameWell'].nunique()

In [ ]:
_df_name_is_cempumppr = _pd_witsml_flattened[_pd_witsml_flattened['description'] == 'CEMPUMPPR']

In [ ]:
_df_name_is_cempumppr.sort_values(by=['startDateTimeIndex']).head(80)

In [ ]:
len(_df_name_is_cempumppr)

In [ ]:
len(_df_name_is_cempumppr['nameWellbore'].unique())

## find sensorid to base real time data on

In [ ]:
_df_name_is_bitdep = _pd_witsml_flattened[_pd_witsml_flattened['description'] == 'BITDEP']

In [ ]:
_df_name_is_bitdep[_df_name_is_bitdep['nameWellbore'] == 'NO 16/2-D-18']

## set sensorid

In [ ]:
_sel_sensorid = int(_df_name_is_bitdep[_df_name_is_bitdep['nameWellbore'] == 'NO 16/2-D-15'].id.iloc[0])
_sel_sensorid 

#### make list of all sensorids to use in panel

In [ ]:
_sensorids_list = pd.Series(_df_name_is_bitdep['nameWellbore']).unique().tolist()
# _sensorids_list

In [ ]:
sr2 = pd.Series(pd.Series(_df_name_is_bitdep['nameWellbore']).tolist(), index = _df_name_is_bitdep['id'])
d = {key: value for key, value in sr2.items()}
_sensorids_list = d
_sensorids_list


#### panel choose _sel_sensorid

In [ ]:
# indicator = pn.indicators.LoadingSpinner(value=False, size=25)
# text = pn.widgets.TextInput(value='Start')

# def update_indicator(event, clicks):
#     if not event:
#         return
    
#     indicator.value = not indicator.value
#     text.value = f"you have clicked me {clicks} times"

# pn.bind(update_indicator, test_widget, test_widget.param.clicks, watch=True)

# pn.Column(pn.Row(test_widget, text),indicator)

In [ ]:
# _sensorids_list = [{1:"fefe"}, {2 :"foo"}]

In [ ]:
sr2 = pd.Series(pd.Series(_df_name_is_bitdep['nameWellbore']).tolist(), index = _df_name_is_bitdep['id'])
d = {key: value for key, value in sr2.items()}
_sensorids_list = d
_sensorids_list



_sel_sensorid_widget = pn.widgets.Select(name='Select', options=_sensorids_list)



# def _update__sel_sensorid
# @param.depends(_sel_sensorid_widget.param.values().__getattribute__('value'))
def _update__sel_sensorid( self, value):
    _sel_sensorid = value
    return _sel_sensorid

# bind _sel_sensorid to event _sel_sensorid_widget and display together with _sel_sensorid_widget
pn.Row(_sel_sensorid_widget, _sel_sensorid_widget.param.values().__getitem__('value'), pn.bind(_update__sel_sensorid, _sel_sensorid_widget,value=_sel_sensorid_widget.param.values().__getitem__('value'), watch=True )).servable() 

# pn.Row(_sel_sensorid_widget, pn.bind(_update__sel_sensorid, _sel_sensorid_widget,value=_sel_sensorid_widget.param.values().items())).servable()

In [ ]:
_sel_sensorid =_sel_sensorid_widget.param.values().__getitem__('value')
_sel_sensorid
## TODO why is _sel_sensorid not updating automatically?

In [ ]:
_sel_sensorid

In [ ]:
_sel_sensorid = 746172675242144

## TIME SERIES QUERY based on _sel_sensorid

In [ ]:
from cognite.client.utils import MIN_TIMESTAMP_MS, MAX_TIMESTAMP_MS

In [ ]:
client.time_series.data.retrieve(id=746172675242144,
    start=MIN_TIMESTAMP_MS,
    end=MAX_TIMESTAMP_MS + 1)

In [ ]:
_df_real_time = client.time_series.data.retrieve(
    id=_sel_sensorid,
    start=MIN_TIMESTAMP_MS,
    end=MAX_TIMESTAMP_MS + 1) 
_df_real_time


## to pandas

In [ ]:
_pd_real_time = _df_real_time.to_pandas( )
# make Value columsn or easier filtering
_pd_real_time['Value'] = _pd_real_time[_pd_real_time.columns]

In [ ]:
_pd_real_time.sort_index(ascending=True)

In [ ]:
_pd_real_time

In [ ]:

_pd_real_time_plot = _pd_real_time
# da['new_index'] = str(da.index)
# da = da.drop(columns=['dw:witsml:log.1ac57fe7-4c90-4bc7-a350-8d17f077bb32.05424d07-6a7d-4ef2-acc1-2c2a6c2eb062.05424d07-6a7d-4ef2-acc1-2c2a6c2eb062.bitdep||v1'])
# da.set_index('new_index')

In [ ]:
_pd_real_time_plot = _pd_real_time_plot.drop(_pd_real_time_plot.columns[[0]], axis=1)  # _pd_witsml_flattened.columns is zero-based pd.Index
_pd_real_time_plot['_new_index'] = _pd_real_time_plot.index
_pd_real_time_plot

In [ ]:
# from bokeh.sampledata.sea_surface_temperature import sea_surface_temperature as sst

# sst.hvplot()

In [ ]:
min(_pd_real_time_plot['_new_index'])
# Timestamp('2022-01-26 07:42:42')

In [ ]:
max(_pd_real_time_plot['_new_index'])

In [ ]:
_pd_real_time_plot[pd.isnull(_pd_real_time_plot).any(axis=1)]

## plot real time data

In [ ]:
_pd_real_time_plot.hvplot(height=1000,
    width=500, title='Real Time',
    x='Value',
    y=['_new_index'], 
    ylim=(min(_pd_real_time_plot['_new_index']), max(_pd_real_time_plot['_new_index']))
).opts(invert_yaxis=True)

## plot with panel

In [ ]:
# sr2 = pd.Series(pd.Series(_pd_witsml_flattened['id']).tolist(), index = _pd_witsml_flattened['id'])
# d = {key: value for key, value in sr2.items()}
# _sensorids_list = d
_sensorids_list = pd.Series(_pd_witsml_flattened['id']).tolist()



# create the Select widget
_sel_sensorid_widget = pn.widgets.Select(name='Select', options=_sensorids_list, width=200)



# def _update__sel_sensorid
# @param.depends(_sel_sensorid_widget.param.values().__getattribute__('value'))
def _update__sel_sensorid(value):
    _sel_sensorid = value
    # do something with _sel_sensorid
    return _sel_sensorid

# bind _sel_sensorid to event _sel_sensorid_widget and display together with _sel_sensorid_widget
# pn.Row(_sel_sensorid_widget, _sel_sensorid_widget.param.values().__getitem__('value'), pn.bind(_update__sel_sensorid, _sel_sensorid_widget,value=_sel_sensorid_widget.param.values().__getitem__('value'), watch=True )).servable() 

# pn.Row(_sel_sensorid_widget, pn.bind(_update__sel_sensorid, _sel_sensorid_widget,value=_sel_sensorid_widget.param.values().items())).servable()

#### make plot

In [ ]:

# make plot of real time data
plot =_pd_real_time_plot.hvplot(height=1000,
    width=500, title='Real Time',
    x='Value',
    y='_new_index', 
    ylim=(min(_pd_real_time_plot['_new_index']), max(_pd_real_time_plot['_new_index']))
).opts(invert_yaxis=True)




#### show panel

In [ ]:

# bind the update function to the widget
_sel_sensorid_widget.param.watch(_update__sel_sensorid, 'value')

# display the widget and plot side by side
pn.Row(_sel_sensorid_widget, plot,_sel_sensorid_widget.value).servable()

## trying with plotly

In [ ]:
pn.pane.Perspective(
    _pd_real_time,columns=['Value']
).servable()

## trying to bind properly

In [ ]:
# pn.extension(design='material')
 
# csv_file = ("https://raw.githubusercontent.com/holoviz/panel/main/examples/assets/occupancy.csv")

# data = pd.read_csv(csv_file, parse_dates=["date"], index_col="date")
 
# data.tail()
 
 
 
# def transform_data(variable, window, sigma):
#     ''' Calculates the rolling average and the outliers '''
#     avg = data[variable].rolling(window=window).mean()
#     residual = data[variable] - avg
#     std = residual.rolling(window=window).std()
#     outliers = np.abs(residual) > std * sigma
#     return avg, avg[outliers]
 
def create_plot(df):
    ''' Plots data '''
    # avg, highlight = transform_data(variable, window, sigma)
    plot = df.hvplot(height=1000,
        width=500, title='Real Time',
        x='Value',
        y=['_new_index'], 
        ylim=(min(df['_new_index']), max(df['_new_index']))
        ).opts(invert_yaxis=True)
    return plot
    # return avg.hvplot(height=300, width=400, legend=False) * highlight.hvplot.scatter(
    #     color="orange", padding=0.1, legend=False
    # )
 
 
 
 
variable_widget = pn.widgets.Select(name="variable", options=list(_pd_real_time_plot.columns))
window_widget = pn.widgets.IntSlider(name="window", value=30, start=1, end=60)
sigma_widget = pn.widgets.IntSlider(name="sigma", value=10, start=0, end=20)
test_widget = pn.widgets.Button(name="testing", button_type="primary")
 
 
# variable_widget.servable()
 
 
 
# create_plot(data=_pd_real_time_plot, window=20, sigma=10)
 
 
 
indicator = pn.indicators.LoadingSpinner(value=False, size=25)
text = pn.widgets.TextInput(value='Start')
 
def update_indicator(event, clicks):
    if not event:
        return
   
    indicator.value = not indicator.value
    text.value = f"you have clicked me {clicks} times"
 
# pn.bind(update_indicator, test_widget, test_widget.param.clicks, watch=True)
 
# pn.Column(pn.Row(test_widget, text),indicator)
 
 
 
 
 
 
# bound_plot = pn.bind(create_plot, variable=variable_widget, window=window_widget, sigma=sigma_widget)
 
 
 
 
 
# first_app = pn.Column(variable_widget, window_widget, test_widget, sigma_widget, bound_plot)
# first_app.servable()

In [ ]:
#plot data 
def create_plot(df, wellbore, sensortype, df_witsml):
    ''' Plots data '''
    # avg, highlight = transform_data(variable, window, sigma)
    # id, df = _make_pd_real_time_plot(_get_sensorid(_pd_witsml_flattened))
    # df = _new_plot
    _w_wellbores_rx = _w_wellbores_select(df_witsml).rx()
    
    plot = df.hvplot(height=1000,
        width=500, title= 'Real Time data for ' + wellbore + " " + sensortype,
        x='Value',
        y=['_new_index'], 
        ylim=(min(df['_new_index']), max(df['_new_index']))#,
        
        #widgets={'nameWellbore' : _w_wellbores}
        # ).opts(invert_yaxis=True)
    )
    
    _pn_plot = pn.Row(pn.Column(_w_wellbores_rx ),pn.rx(plot))
    return plot
    # return avg.hvplot(height=300, width=400, legend=False) * highlight.hvplot.scatter(
    #     color="orange", padding=0.1, legend=False
    # )





# make datasets
def _make_pd_real_time_plot(sensorid) :
    """
    make pd of real time dat to be plotted for given sensorid
    Args: sensorid - the id of the sensor
    Returns: the id column and the pandas to be plotted
    """
    _pd_real_time_plot = _make_pd_real_time(sensorid)
    _realtimedata_id = _pd_real_time_plot.columns
    _pd_real_time_plot = _pd_real_time_plot.drop(_pd_real_time_plot.columns[[0]], axis=1)  # _pd_witsml_flattened.columns is zero-based pd.Index
    _pd_real_time_plot['_new_index'] = _pd_real_time_plot.index
    return _realtimedata_id, _pd_real_time_plot

def _make_pd_real_time(sensorid):
    """make pd of real time data for given sensorid"""
    _pd_real_time = _get_df_real_time(sensorid).to_pandas()
    # make Value columsn or easier filtering
    _pd_real_time['Value'] = _pd_real_time[_pd_real_time.columns]
    return _pd_real_time

def _get_df_real_time(sensorid):
    """make cognite client of real time data for given sensorid"""
    _df_real_time = client.time_series.data.retrieve(
        id=sensorid, # TODO make function to choose
        start=MIN_TIMESTAMP_MS,
        end=MAX_TIMESTAMP_MS + 1) 
    return _df_real_time   

# get sensor id
def _get_sensorid(df):
    """
        Args: df on the format of _pd_witsml_flattened
        Returns: the sensor id form the _pd_witsml_flattened based on chosen wellbore and chosen sensortype
    """
    # _chosen_wellbore = str(_w_wellbores_select(df).value)
    _chosen_wellbore = _w_wellbores_select(df).value
    _chosen_sensortype= str(_w_sensortypes_select(df).value)
    # _chosen_sensortype= _chosen_sensortype.value
    _sensor_row = df[(df['nameWellbore'] == _chosen_wellbore) & (df['uid'] == _chosen_sensortype)]
    _sensorid = int(_sensor_row['id'].iloc[0])
    _sensortype = _sensor_row['uid'].iloc[0]
    _wellbore = _sensor_row['nameWellbore'].iloc[0]
    return _sensorid, _wellbore, _sensortype




# make list for the widgets

# def _make_sensorids_list(df):
#     sr2 = pd.Series(pd.Series(df['id']).tolist(), index = df['id'])
#     d = {key: value for key, value in sr2.items()}
#     _sensorids_list = d
#     return _sensorids_list


def _make_wellbores_list(df):
    # sr2 = pd.Series(pd.Series(df['nameWellbore']).unique().tolist(), index = df['id'])
    _wellbores_list = pd.Series(df['nameWellbore']).unique().tolist()
    # d = {key: value for key, value in sr2.items()}
    # _sensorids_list = d
    return _wellbores_list

def _make_sensortypes_list(df):
    # sr2 = pd.Series(pd.Series(df['uid']).unique().tolist(), index = df['id']) # TODO check that uid is same as name aka sensortype
    _sensortypes_list = pd.Series(df['uid']).unique().tolist()
    # d = {key: value for key, value in sr2.items()}
    # _sensorids_list = d
    return _sensortypes_list




## make widgets
def _w_wellbores_select(df):
    """
    Args: df on the format of _pd_witsml_flattened
    """
    _sel_sensorid_widget = pn.widgets.Select(name='Select', options=_make_wellbores_list(df))
    return _sel_sensorid_widget

def _w_sensortypes_select(df):
    """
    Args: df on the format of _pd_witsml_flattened
    """
    _sel_sensortype_widget = pn.widgets.Select(name='Select', options=_make_sensortypes_list(df))
    return _sel_sensortype_widget

# get widget values
def _get_wellbore():
    return _w_wellbores_select().value

def _get_sensortype():
    return _w_sensortypes_select().value

# @pn.depends(_w_wellbores_select(_pd_witsml_flattened))
# def _new_plot():
    # return _w_wellbores_select(_pd_witsml_flattened).value

def b(event):
    print("hellp")



# define the update function
def update_plot(event):
    sensorid = _get_sensorid(_pd_witsml_flattened)
    id, data = _make_pd_real_time_plot(sensorid)
    new_plot = create_plot(data, _pd_witsml_flattened)
    plot.object = new_plot.object

# _w_wellbores_select(df)



# _w_wellbores_select =  select(_pd_witsml_flattened)
# _w_sensortypes_select = pn.widgets.Select(name='Select Sensor Type')

# define the update function
def update_plot(wellbore):
    sensorid = _get_sensorid(_pd_witsml_flattened, wellbore)
    id, data = _make_pd_real_time_plot(sensorid)
    new_plot = create_plot(data, wellbore, _w_sensortypes_select.value, _pd_witsml_flattened)
    plot.object = new_plot.object

# bind the update function to the widget values
_w_wellbores_select.param.value = pn.bind(update_plot, wellbore=_w_wellbores_select.value)

_w_sensortypes_select.param.value = pn.bind(update_plot, wellbore=_w_wellbores_select.value)

# display the widgets and plot
pn.Row(_w_wellbores_select, _w_sensortypes_select, plot).servable()



# create the Select widgets
# _w_wellbores_select_widget = _w_wellbores_select(_pd_witsml_flattened)
# _w_sensortypes_select_widget = pn.widgets.Select(name='Select Sensor Type', options=_make_sensortypes_list(_pd_witsml_flattened))

# bind the update function to the widget values
#_w_wellbores_select(_pd_witsml_flattened).param.watch(update_plot, 'value')
# pn.bind(update_plot, pn.widgets.Select(name='Select Wellbore', options=_make_wellbores_list(_pd_witsml_flattened)))
# _w_sensortypes_select.param.watch(update_plot, 'value')

# \pn.bind(update_plot, event='new')
# pn.bind(update_plot, event='new')

# pn.bind(update_plot, _w_wellbores_select_widget,  watch=True)

# _w_wellbores_select.param.value.bind(update_plot)
# _w_sensortypes_select.param.value.bind(update_plot)

In [ ]:
#### REWQRITE PROPER NEW ONES


#plot data 
def create_plot(_plot_param):#, wellbore, sensortype, df_witsml):
    ''' Plots data '''    
    plot = _plot_param.hvplot(height=1000,
        width=500, title= 'Real Time data for ' , #+ wellbore + " " + sensortype,
        x='Value',
        y=['_new_index'] 
        # ,ylim=(min(df['_new_index']), max(df['_new_index']))#,
    )
    
    return plot

# make datasets
def _make_pd_real_time_plot(sensorid) :
    """
    make pd of real time dat to be plotted for given sensorid
    Args: sensorid - the id of the sensor
    Returns: the id column and the pandas to be plotted
    """
    _pd_real_time_plot = _make_pd_real_time(sensorid)
    _realtimedata_id = _pd_real_time_plot.columns
    _pd_real_time_plot = _pd_real_time_plot.drop(_pd_real_time_plot.columns[[0]], axis=1)  # _pd_witsml_flattened.columns is zero-based pd.Index
    _pd_real_time_plot['_new_index'] = _pd_real_time_plot.index
    return _realtimedata_id, _pd_real_time_plot

def _make_pd_real_time(sensorid):
    """make pd of real time data for given sensorid"""
    _pd_real_time = _get_df_real_time(sensorid).to_pandas()
    # make Value columsn or easier filtering
    _pd_real_time['Value'] = _pd_real_time[_pd_real_time.columns]
    return _pd_real_time

def _get_df_real_time(sensorid):
    """make cognite client of real time data for given sensorid"""
    _df_real_time = client.time_series.data.retrieve(
        external_id= None,
        id=sensorid, # TODO make function to choose
        start=MIN_TIMESTAMP_MS,
        end=MAX_TIMESTAMP_MS + 1) 
    return _df_real_time   

# get sensor id
def _get_sensorid(df, wellbore, sensortype):
    """
        Args: df on the format of _pd_witsml_flattened
        Returns: the sensor id form the _pd_witsml_flattened based on chosen wellbore and chosen sensortype
    """
    _sensor_row = df[(df['nameWellbore'] == wellbore) & (df['uid'] == sensortype)]
    _sensorid = _sensor_row['id'].iloc[0]
    return _sensorid


# make lists to use in widgets
def _make_wellbores_list(df):
    _wellbores_list = pd.Series(df['nameWellbore']).unique().tolist()
    return _wellbores_list

def _make_sensortypes_list(df):
    _sensortypes_list = pd.Series(df['uid']).unique().tolist()
    return _sensortypes_list




## make widgets
def _w_wellbores_select(df):
    """
    Args: df on the format of _pd_witsml_flattened
    """
    _sel_sensorid_widget = pn.widgets.Select(name='Select', options=_make_wellbores_list(df))
    return _sel_sensorid_widget

def _w_sensortypes_select(df):
    """
    Args: df on the format of _pd_witsml_flattened
    """
    _sel_sensortype_widget = pn.widgets.Select(name='Select', options=_make_sensortypes_list(df))
    return _sel_sensortype_widget


In [ ]:
# display the widgets and plot
# pn.Row(_w_wellbores_select, _w_sensortypes_select, plot).servable()

In [ ]:

sensorid, wellbore, sensor =  _get_sensorid(_pd_witsml_flattened)
id, data = _make_pd_real_time_plot(sensorid)


# #btn = pn.widgets.Button(name = "click me", button_type = 'primary')

plot = create_plot(data, wellbore, sensor, _pd_witsml_flattened)
# plot.param.trigger('draw')
plot

In [ ]:
import panel as pn
import pandas as pd
import xarray as xr
import hvplot.xarray

# create the Select widgets
_w_wellbores_select = pn.widgets.Select(name='Select Wellbore',  options=_make_wellbores_list(_pd_witsml_flattened))
_w_sensortypes_select = pn.widgets.Select(name='Select Sensor Type')

# create the DataFrame and plot
_pd_real_time_plot = pd.DataFrame({
    'Value': [1, 2, 3, 4, 5],
    '_new_index': pd.date_range('2022-01-01', periods=5, freq='D')
})

plot = _pd_real_time_plot.hvplot(height=1000,
    width=500, title='Real Time',
    x='Value',
    y=['_new_index'], 
    ylim=(min(_pd_real_time_plot['_new_index']), max(_pd_real_time_plot['_new_index']))
).opts(invert_yaxis=True)

# define the update function
def update_plot(event):
    sensorid = _get_sensorid(_pd_witsml_flattened)
    id, data = _make_pd_real_time_plot(sensorid)
    new_plot = create_plot(data, _pd_witsml_flattened)
    plot.object = new_plot.object

# bind the update function to the widget values
_w_wellbores_select.param.watch(update_plot, 'value')
_w_sensortypes_select.param.watch(update_plot, 'value')

# display the widgets and plot
pn.Row(_w_wellbores_select, _w_sensortypes_select, plot).servable()

In [ ]:
import pandas as pd
import numpy as np
import hvplot.pandas
import panel as pn

# create dataset d1
d1 = pd.DataFrame({
    'x': np.arange(10),
    'y1': np.random.rand(10),
    'y2': np.random.rand(10)
})

# create dataset d2 with different column names
d2 = pd.DataFrame({
    'x': np.arange(10),
    'z1': np.random.rand(10),
    'z2': np.random.rand(10)
})

# plot d1 using hvplot
plot = d1.hvplot.line(x='x', y=['y1', 'y2'])

# define the update function
def update_plot(event):
    #global plot
    newplot = d2.hvplot.line(x='x', y=['z1', 'z2'], title="d2")
    plot = newplot
    plot

# create the button widget
button = pn.widgets.Button(name='Update Plot')


indicator = pn.indicators.LoadingSpinner(value=False, size=25)
text = pn.widgets.TextInput(value='Start')
 
def update_indicator(event, clicks):
    if not event:
        return
   
    indicator.value = not indicator.value

    test = pn.Column(button, (d2.hvplot.line(x='x', y=['z1', 'z2'], title="d2")), indicator, text).servable()
    test.servable
    text.value = f"you have clicked me {clicks} times"
    plot.title = "hjhkj"
 
# pn.bind(update_indicator, button, button.param.clicks, watch=True)

# pn.bind(update_plot, button, watch=True)

# bind the update function to the button click event
# button.on_click(update_plot)

# display the button and plot using Panel
# pn.Column(button, plot, indicator, text).servable()
hvplot.bind(update_plot, button).interactive().hvplot.line(title="d2")


In [ ]:
import hvplot.pandas  # noqa
import pandas as pd
import panel as pn

pn.extension()


def function_to_generate_dataframe(x):
    # x only used to trigger the function
    df =  pd.DataFrame({
    'x': np.arange(10),
    'y1': np.random.rand(10),
    'y2': np.random.rand(10)
})
    # df["C"] = df["C"].round()
    return df


widget = pn.widgets.IntSlider(end=100)
hvplot.bind(function_to_generate_dataframe, widget).interactive().hvplot.line()


In [ ]:
import hvplot.pandas
import pandas as pd
import panel as pn
import numpy as np

pn.extension()

# create dataset d1
d1 = pd.DataFrame({
    'x': np.arange(10),
    'y1': np.random.rand(10)
})


# create dataset d2 with different column names
d2 = pd.DataFrame({
    'x': np.arange(10),
    'y1': np.random.rand(10)
    })

def function_to_generate_dataframe(x):
    # x only used to trigger the function
    global d1, d2
    if x == 'd1':
        df = d1
    else:
        df = d2
    
    return df.hvplot.line(x='x', title=x)

widget = pn.widgets.Select(options=['d1', 'd2'])
hvplot.bind(function_to_generate_dataframe, widget).interactive().hvplot.line()

In [ ]:
#WORKS


# create dataset d1
d1 = _pd_real_time_plot


# create dataset d2 with different column names
d2 = _pd_real_time_plot

def function_to_generate_dataframe2(x):
    # x only used to trigger the function
    global d1, d2
    if x == 'd1':
        df = d1
    else:
        df = d2
    
    return df.hvplot.line(x='Value', title=x)

widget2 = pn.widgets.Select(options=['d1', 'd2'])
p = hvplot.bind(function_to_generate_dataframe2, widget2).interactive()
p.hvplot().line()


In [ ]:
d2

In [ ]:
#tesing with new code and proper WORRKSSS YESSS
# import hvplot.pandas
def function_to_generate_dataframe(wellbore, sensortype):
    print(wellbore, sensortype)
    sensorid = _get_sensorid(_pd_witsml_flattened,wellbore,sensortype)
    print(type(sensorid))
    print(sensorid)
    s= int(sensorid)
    id, data = _make_pd_real_time_plot(s)
    return data

plot = hvplot.bind\
    (function_to_generate_dataframe, _w_wellbores_select(_pd_witsml_flattened),_w_sensortypes_select(_pd_witsml_flattened)) \
    .interactive()
plot.head(20)



In [ ]:
create_plot(plot)

In [ ]:
d2

In [ ]:
_w_wellbores_select(_pd_witsml_flattened)

In [ ]:
_w_wellbores_select(_pd_witsml_flattened).param.values()

In [ ]:


df = pn.rx(data)
plot = df.hvplot(height=1000,
    width=500, title='Real Time',
    x='Value',
    y=['_new_index'], 
    ylim=(min(df['_new_index']), max(df['_new_index']))    # ).opts(invert_yaxis=True)
)



In [ ]:
df = data
e = pn.widgets.IntSlider(value=2, start=1, end=10, name="e").rx()
pn.Row(pn.rx(df.hvplot.line)(line_width=e))

In [ ]:
min(data['_new_index'])

In [ ]:
create_plot(data, _pd_witsml_flattened)

In [ ]:
print(wellbore, sensor)

In [ ]:
_get_sensorid(_pd_witsml_flattened)

In [ ]:
_pd_witsml_flattened[(_pd_witsml_flattened['nameWellbore'] == 'NO 16/2-D-18') & (_pd_witsml_flattened['uid'] == 'BITDEP')]


In [ ]:
_pd_witsml_flattened[_pd_witsml_flattened['id'] == 881650803765]

In [ ]:
_pd_witsml_flattened

In [ ]:
_sel_sensorid

In [ ]:
id, data = _make_pd_real_time_plot(7119861774595595)
data

In [ ]:
test= pn.rx(_pd_real_time_plot)

head_widget = pn.widgets.IntSlider(name="Head", start=1, end=10)

test.head(head_widget)

In [ ]:

pn.ReactiveExpr(
    test.head(head_widget),  # only show a few rows to save some space
    widget_layout=pn.Column,
    widget_location="top",
)

In [ ]:
table = pn.widgets.Tabulator(test, page_size=10, pagination="remote")
pn.Column(_w_wellbores_select(_pd_witsml),  table)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import streamlit as st

from matplotlib.figure import Figure

data = np.random.normal(1, 1, size=100)
fig = Figure(figsize=(8,4))
ax = fig.subplots()
bins = pn.widgets.slider(value=20, min_value=10, max_value=30, step=1, label="Bins")
ax.hist(data, bins=bins)

st.pyplot(fig)